The data set choosen for following analysis is related to Spam & no-spam (Ham) e-mails. This again looks very familiar to every one as almost everyone deals with such kind of problems when matter is related to e-mail exchanges . You may remember that in addition to standard screening from e-mail service providers , sometime we still get some e-mail which are spams. It can be easily corelated by the outcome of the analysis below. The thing to be focused for that matter is "confusion matrix", which we get as an outcome of the analysis.





In [ ]:
from IPython.display import Image
import os
("../input/images/Spam ham emoji.JPG")



In [ ]:
Image("../input/images/Spam ham emoji.JPG")

This a classification problem and its analysis can be done by following two ways. 

    1) Logistic regression
    2) Naive Nayes Theorem
    
As this problem involves words dictionaries, we will use Naive Bayes as it is best fit for such kind of analysis.
There are two sub analysis methods under Naive bayes header as follows;

    1) Multinomial Naive Bayes 
    2) Bernoullis Naive bayes
    
In general Bernoullis Naive Baive classifier is known to be more efficient for categorical classification problems. I will use Bernoullis Naive Bayes for My analysis.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
#spam_ham= pd.read_csv("../input/heart.csv")

Spam-ham Data set loading from repository

In [ ]:
spam_ham= pd.read_csv("../input/spam-or-not-spam-dataset/spam_or_not_spam.csv")
spam_ham.head()

Additional detail checks abour data frame

In [ ]:
spam_ham.info()

In [ ]:
spam_ham.describe()

Checking the Nan or null entries for e-mail

In [ ]:
spam_ham['email'].isnull().sum()

Lets drop the row with null entry

In [ ]:
spam_ham= spam_ham.dropna(how='any',axis=0)
spam_ham.info()

**Plotting Spam ham count**

In [ ]:
sns.countplot(x='label', data = spam_ham)
plt.title('Number of Spam (1) & ham (0) from e-mail dataset ')
plt.show()

In [ ]:
spam_ham['label'].sum()

Writing a small function for counting number of '0' & '1' in a column

In [ ]:
def sum_func(df,column): # this function will help in counting the same group entries
    for i in range(len(column)):
        count = df[column].value_counts()
        return count

sum_func(spam_ham, 'label')
    


In [ ]:
print( "Spam percentage is ",spam_ham['label'].sum()/len(spam_ham.index)* 100, "%")

**Prepare data for ML modelling**

In [ ]:
numpy_array = spam_ham.as_matrix()
X= spam_ham.email
y= spam_ham.label
#X=numpy_array[:,0]
#y=numpy_array[:,1]
#y = y.astype('int')
print("X")
print(X)
print("y")
print(y)

**Preparing the data for ML modelling**

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=100)
X_train

**Vectorizing the sentences and dropping the stop words**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect= CountVectorizer(stop_words="english")
vect.fit(X_train)

# printing the Vocabulary texts

print(vect.vocabulary_)

In [ ]:
X_train.shape

In [ ]:
X_train_transformed= vect.transform(X_train)
X_test_transformed = vect.transform(X_test)
#print(X_train_transformed)
print(X_test_transformed)

In [ ]:
#X = X_transformed.toarray()
#X

**Modelling using Naive Bayes**

In [ ]:
#X_train_transformed=X_transformed.toarray()
X_train_transformed

In [ ]:
X_train_transformed.shape

In [ ]:
# converting matrix to dataframe
#pd.DataFrame(X_train_transformed, columns=vect.get_feature_names())

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb=BernoulliNB()
bnb.fit(X_train_transformed, y_train)
proba= bnb.predict_proba(X_test_transformed)
y_pred= bnb.predict(X_test_transformed)

#Converting array to data frame
#proba = pd.DataFrame(proba)
#proba.tail()
# printing the overall accuracy
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

* The Accuracy of above model is 89.2 %

**Confusion matrix evaluation of above Model**

In [ ]:
from sklearn import metrics
confusion= metrics.confusion_matrix(y_test, y_pred)
print(confusion)

Above confusion matrix Score shows that the model prediction will lead to loosing 66 hams due to wrong prediction as spam. This requires further refinement

In [ ]:
pd.DataFrame(proba, columns=['Ham','Spam'])
#pd.DataFrame(proba)

**Calculation of Sensitivity, specificity**

In [ ]:
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
TP = confusion[1, 1]

In [ ]:
sensitivity = TP/float(TP+FN)
print("Sensitivity= ",sensitivity)
specificity= TN/float(TN+FP)
print("Specificity= ", specificity)
print("Precision= ", TP/float(TP+FP))

print("PRECISION SCORE :",metrics.precision_score(y_test, y_pred))
print("RECALL SCORE :", metrics.recall_score(y_test, y_pred))
print("F1 SCORE :",metrics.f1_score(y_test, y_pred))

In [ ]:
# creating an ROC curve
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, proba[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)

# matrix of thresholds, tpr, fpr
print(pd.DataFrame({'Threshold': thresholds,
              'TPR': true_positive_rate,
              'FPR':false_positive_rate
             }))

# plotting the ROC curve


plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], [0, 1], 'k--')
plt.show()

### Bernoullis NB modelling gave better result

In [ ]:
roc_auc = auc(false_positive_rate, true_positive_rate)

In [ ]:
print (roc_auc)

In [ ]:
print(max(thresholds))

In [ ]:
from sklearn.metrics import precision_recall_curve 

precision, recall, thresholds = precision_recall_curve(y_test, proba[:,1])
# create plot
plt.plot(recall, precision, label='Precision-recall curve')

_ = plt.xlabel('Recall')
_ = plt.ylabel('Precision')
_ = plt.title('Precision-recall curve')
_ = plt.legend(loc="lower left")

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')

From above Curve it appears as Precision Starts at around .3 and at this point based on the curve FP(False Positive) is higer and as we move right it reduces and leads to increase in TP(True Positive counts).

On the same curve at 0.3 Precison recall is maximum . Which means FN (False negative) is minimum.


**Checking the area under Precision-Recall Curve**

In [ ]:
from sklearn.metrics import average_precision_score
average_precision_score(y_test, proba[:, 1])

The output of the Precison_Recall classifier is 0.7. This is also called as Average Precision. The more it is closer to 1, better is the Classifier.

**Conlusion**

To further describe it in practical sense, it means at Precision 0.3 according the above Precision recall curve, the Model will allow higher hams to be classified as Spam and most accurately it will predict the spam count. Which is negligible miss of spam classification. But this is not called a good refrence point because we will loose most of the good e-mails(Hams) because of predicted as spam at that threshold.

Here comes the domain knowledge, which helps in making a clear decision based on what is best  based on expected need.

Suppose we do not want to miss larger hams . We need to come to point where we have to allow some compromise . Which will lead to some cross classification.

Bsed on the above curve as we move right we increase the presion with the cost of loss of Recall. Still we can claim 70 % precision with 80 % recall based on above ML classifier.

More importantly, if we look at the data distribution for the classes, it seems to be bit imbalance i.e. only 16.63 % data is classified as spam and rest is ham. So to evaluate a ML model performance for such cases, Precision-Recall curve makes more sence as compared to ROC curve